In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def calculate_high_low_range(ticker_symbol, periods=[5, 10]):
    """
    Calculate the average high-to-low range in percent for a stock
    over specified trading sessions.

    Parameters:
    ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
    periods (list): List of periods to calculate (default: [5, 10])

    Returns:
    dict: Dictionary with average ranges for each period
    """

    # Download stock data (get more days to ensure we have enough trading days)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)  # Get 30 calendar days to ensure enough trading days

    print(f"Fetching data for {ticker_symbol}...")
    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)

    if stock_data.empty:
        print(f"No data found for ticker: {ticker_symbol}")
        return None

    # Calculate daily high-to-low range as a percentage
    stock_data['Range_Percent'] = ((stock_data['High'] - stock_data['Low']) / stock_data['Low']) * 100

    # Display recent data
    print(f"\nRecent trading data for {ticker_symbol}:")
    print(stock_data[['High', 'Low', 'Close', 'Range_Percent']].tail(max(periods)))

    # Calculate average ranges for each period
    results = {}
    for period in periods:
        if len(stock_data) >= period:
            avg_range = stock_data['Range_Percent'].tail(period).mean()
            results[f'{period}_day_avg'] = avg_range
            print(f"\nAverage High-Low Range (Last {period} trading sessions): {avg_range:.2f}%")
        else:
            print(f"\nInsufficient data for {period}-day calculation")
            results[f'{period}_day_avg'] = None

    return results


In [3]:
ticker = "AAPL"  # Example: Apple Inc.

print("=" * 60)
print(f"Stock High-Low Range Analysis")
print("=" * 60)

results = calculate_high_low_range(ticker, periods=[5, 10])

if results:
    print("\n" + "=" * 60)
    print("Summary:")
    print("=" * 60)
    for period, avg_range in results.items():
        if avg_range is not None:
            print(f"{period.replace('_', ' ').title()}: {avg_range:.2f}%")

Stock High-Low Range Analysis
Fetching data for AAPL...


/tmp/ipython-input-198414712.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)



Recent trading data for AAPL:
Price             High         Low       Close Range_Percent
Ticker            AAPL        AAPL        AAPL              
Date                                                        
2025-10-22  262.850006  255.429993  258.450012      2.904911
2025-10-23  260.619995  258.010010  259.579987      1.011583
2025-10-24  264.130005  259.179993  262.820007      1.909874
2025-10-27  269.119995  264.649994  268.809998      1.689024
2025-10-28  269.890015  268.149994  269.000000      0.648898
2025-10-29  271.410004  267.109985  269.700012      1.609831
2025-10-30  274.140015  268.480011  271.399994      2.108166
2025-10-31  277.320007  269.160004  270.369995      3.031655
2025-11-03  270.850006  266.250000  269.049988      1.727702
2025-11-04  271.485992  267.614990  270.649994      1.446482

Average High-Low Range (Last 5 trading sessions): 1.98%

Average High-Low Range (Last 10 trading sessions): 1.81%

Summary:
5 Day Avg: 1.98%
10 Day Avg: 1.81%
